In [ ]:
import numpy as np
import pandas as pd
from glob import glob
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential

In [ ]:
train_path = r'E:\YellowRust\train\train'
test_path = r'E:\YellowRust\test\test'

In [ ]:
IMAGE_SIZE = [224, 224] #Default image size for VGG16
folders = glob(train_path + '/*') #Get number of classes
num_classes = len(folders)
print ('Total Classes = ' + str(num_classes))

Total Classes = 6


In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)
validation_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (224, 224),
                                                 batch_size = 64,
                                                 class_mode = 'categorical')

Found 15000 images belonging to 6 classes.


In [ ]:
test_set = test_datagen.flow_from_directory(test_path,
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 5421 images belonging to 6 classes.


In [ ]:
vgg = VGG19(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [ ]:
for layer in vgg.layers:
  layer.trainable = False

In [ ]:
x = Flatten()(vgg.output) #Output obtained on vgg16 is now flattened.
prediction = Dense(len(folders), activation='softmax')(x)

In [ ]:
model = Model(inputs=vgg.input, outputs=prediction)

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(training_set, validation_data=test_set, epochs=30, batch_size=32)

Epoch 1/30
235/235 [==============================] - 271s 1s/step - loss: 1.3906 - accuracy: 0.4361 - val_loss: 1.0157 - val_accuracy: 0.5683
Epoch 2/30
235/235 [==============================] - 182s 775ms/step - loss: 1.1012 - accuracy: 0.5399 - val_loss: 0.9479 - val_accuracy: 0.6097
Epoch 3/30
235/235 [==============================] - 138s 586ms/step - loss: 1.0166 - accuracy: 0.5850 - val_loss: 0.9171 - val_accuracy: 0.6080
Epoch 4/30
235/235 [==============================] - 152s 648ms/step - loss: 0.9832 - accuracy: 0.5957 - val_loss: 0.9845 - val_accuracy: 0.5809
Epoch 5/30
235/235 [==============================] - 145s 617ms/step - loss: 0.8943 - accuracy: 0.6383 - val_loss: 0.7793 - val_accuracy: 0.6847
Epoch 6/30
235/235 [==============================] - 151s 643ms/step - loss: 0.8367 - accuracy: 0.6595 - val_loss: 0.8791 - val_accuracy: 0.6394
Epoch 7/30
235/235 [==============================] - 153s 652ms/step - loss: 0.8422 - accuracy: 0.6649 - val_loss: 0.7360 - va

In [ ]:
model_path = 'vgg19_model.h5'
model.save(model_path)
print("VGG-19 model saved at:", model_path)

VGG-19 model saved at: vgg19_model.h5
